In [1]:
import os


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [5]:
from typing import Annotated
from typing import Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from typing_extensions import TypedDict
from langgraph.graph import MessagesState, END,StateGraph, START
from langgraph.types import Command
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent


In [6]:
members1=["manage_overloaded_restaurant","damaged_packaging_dispute","communicate"]

In [7]:
options=members1+["FINISH"]

In [8]:
class State(MessagesState):
    next:str

In [10]:
class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""
    next: Literal["manage_overloaded_restaurant","damaged_packaging_dispute","communicate", 'FINISH']

In [12]:
system_prompt=f"""
You are a supervisor agent of grab-food section, tasked with managing a orchestration between the following agents: {members1
}. 

"""

In [15]:
def grab_food(state:State)->Command[Literal["manage_overloaded_restaurant","damaged_packaging_dispute","communicate", 'FINISH']]:
    messages = [{"role": "system", "content": system_prompt},] + state["messages"] # in this message the content will be the if issue is orderring if the preparation time is excess then it will call the overloaded restaurant tool
    if state.next == "manage_overloaded_restaurant":
        messages.append({"role": "user", "content": "The restaurant is overloaded, please manage the situation."})
    elif state.next == "damaged_packaging_dispute":
        messages.append({"role": "user", "content": "There is a dispute regarding damaged packaging."})
    elif state.next == "communicate":
        messages.append({"role": "user", "content": "Please communicate with the customer."})
    else:
        messages.append({"role": "user", "content": "No further action is needed."})
    
    response=llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto=="FINISH":
        goto =END

    return Command(goto=goto,update={"next":goto})

In [17]:
from langchain_core.tools import tool

@tool
def notify_customer(delay_minutes: int) -> str:
    """
    Notify the customer about long wait times and offer a voucher for the inconvenience.
    
    Args:
        delay_minutes: The expected delay in minutes
        
    Returns:
        A message confirming the customer has been notified
    """
    return f"Customer notified about {delay_minutes} minute delay. A voucher has been offered for the inconvenience."

@tool
def reroute_driver(delay_minutes: int) -> str:
    """
    Re-route the driver to a short, nearby delivery while food is being prepared.
    
    Args:
        delay_minutes: The expected delay in minutes
        
    Returns:
        A message confirming the driver has been rerouted
    """
    return f"Driver has been rerouted to a nearby delivery that can be completed within {delay_minutes} minutes while food is being prepared."

@tool
def suggest_alternatives(delay_minutes: int) -> str:
    """
    Find a similar restaurant nearby with a shorter wait time and propose it to the customer.
    
    Args:
        delay_minutes: The expected delay in minutes
        
    Returns:
        A message with alternative restaurant suggestions
    """
    return f"Customer has been provided with 3 similar restaurant alternatives with wait times under {delay_minutes//2} minutes."

def manage_overloaded_restaurant(state: State) -> Command[Literal["grab_food", "FINISH"]]:
    """
    Handle situations where a restaurant is overloaded with orders, causing long wait times.
    Uses tools to notify customer, reroute driver, and suggest alternatives if needed.
    """
    # Create a React agent that uses our tools
    tools = [notify_customer, reroute_driver, suggest_alternatives]
    
    # Create a system message to guide the agent
    system_message = """
    You are a Grab Food order management agent handling an overloaded restaurant situation.
    The kitchen prep time is 40 minutes, which is longer than usual.
    
    Follow these steps:
    1. Notify the customer about the long wait time and offer a voucher
    2. Optimize driver time by re-routing them to another nearby delivery
    3. If the delay is critical, suggest alternative restaurants to the customer
    
    Use the available tools to complete these tasks efficiently.
    """
    
    # Create an agent that can use the tools to handle the overloaded restaurant
    agent = create_react_agent(llm, tools, system_message)
    
    # Add the user query to the messages
    messages = state["messages"] + [
        HumanMessage(content="The restaurant is overloaded with a 40-minute prep time. Handle this situation.")
    ]
    
    # Run the agent
    agent_response = agent.invoke(messages)
    
    # Process the agent's response to determine next steps
    # For simplicity, we'll just check if any action was taken
    if any(tool.name in agent_response.content for tool in tools):
        # If actions were taken, we can move on
        goto = "FINISH" 
    else:
        # If no action was taken, loop back to grab_food
        goto = "grab_food"
    
    if goto == "FINISH":
        goto = END
        
    return Command(goto=goto, update={"next": goto})

In [19]:
@tool
def initiate_mediation_flow():
    """driver or customer will trigger the "at-the-door-resolutionflow then the agent manage_packaging_dispute will initiate this toolit will mainly puuase the order """
    return f"the order has been paused and the dispute management is started"

@tool
def collect_evidence():
    """now the interface will open at the both side once the initiate_mediation_flow started by the agent
    it will prompt the both  parties to provide the photos of damaged packaging andd also this method will prompt both  the user and the driver some dynamic questions such as
    , Driver: "Was the bag sealed
by the merchant?", Customer: "Was the seal intact upon
handover?") """
    return f"the photos and the answers taken from both parties"

@tool
def analyze_evidence():
    """This tool will analyze the evidence provided by both parties and determine the outcome of the dispute. and it will trigger the tools such as issue_instant_refund and exonerate_driver and log_merchant_packaging_feedback"""
    return f"the evidence has been analyzed and a decision has been made"

@tool
def issue_instant_refund():
    """This tool will issue an instant refund to the customer if the dispute is resolved in their favor."""
    return f"an instant refund has been issued to the customer"

@tool
def exonerate_driver():
    """This tool will exonerate the driver if the evidence shows that they are not at fault."""
    return f"the driver has been exonerated"

@tool
def log_merchant_packaging_feedback():
    """This tool will log the merchant's feedback regarding the packaging."""
    return f"the merchant's packaging feedback has been logged and it will warn them to manage the packaging better"

@tool
def notify_resolution():
    """This tool will notify both parties about the outcome of the dispute."""
    return f"both parties have been notified about the outcome of the dispute"

def damaged_packaging_dispute(state: State) -> Command[Literal["grab_food", "FINISH"]]:
    """
    Handle disputes related to damaged packaging during food delivery.
    Uses a sequence of tools to mediate between customer and driver, collect evidence,
    analyze the situation, and resolve the dispute fairly.
    """
    # Collect all the tools needed for packaging dispute resolution
    tools = [
        initiate_mediation_flow,
        collect_evidence,
        analyze_evidence,
        issue_instant_refund,
        exonerate_driver,
        log_merchant_packaging_feedback,
        notify_resolution
    ]
    
    # Create a system message to guide the agent
    system_message = """
    You are a Grab Food dispute resolution agent handling a damaged packaging situation.
    
    Follow these steps in sequence:
    1. Initiate the mediation flow to pause the order and start dispute management
    2. Collect evidence from both the customer and driver (photos and answers to specific questions)
    3. Analyze the evidence to determine who is at fault
    4. Based on analysis:
       - Issue a refund to the customer if appropriate
       - Exonerate the driver if they're not at fault
       - Log feedback for the merchant about packaging quality
    5. Notify all parties about the resolution
    
    Use the available tools to complete these tasks in the correct order.
    """
    
    # Create an agent that can use the tools to handle the packaging dispute
    agent = create_react_agent(llm, tools, system_message)
    
    # Add the user query to the messages
    messages = state["messages"] + [
        HumanMessage(content="There is a dispute regarding damaged packaging. The customer claims the food was spilled in the bag upon arrival.")
    ]
    
    # Run the agent
    agent_response = agent.invoke(messages)
    
    # Process the agent's response to determine next steps
    # We'll check if the resolution was notified, which should be the last step
    if "notify_resolution" in agent_response.content:
        # If the dispute was fully resolved, we can finish
        goto = "FINISH"
    else:
        # If the resolution isn't complete, go back to grab_food for further processing
        goto = "grab_food"
    
    if goto == "FINISH":
        goto = END
        
    return Command(goto=goto, update={"next": goto})

In [21]:
# Create the workflow graph
graph = StateGraph(State)

# Add all nodes to the graph
graph.add_node("grab_food", grab_food)
graph.add_node("manage_overloaded_restaurant", manage_overloaded_restaurant)
graph.add_node("damaged_packaging_dispute", damaged_packaging_dispute)
graph.add_node("communicate", communicate)

NameError: name 'communicate' is not defined

In [22]:
# Set up the connections
# The grab_food node can route to any of the specialized agents or directly to END
graph.add_edge("grab_food", "manage_overloaded_restaurant")
graph.add_edge("grab_food", "damaged_packaging_dispute")
graph.add_edge("grab_food", "communicate")
graph.add_edge("grab_food", END)

# Each specialized agent routes back to grab_food or to END
graph.add_edge("manage_overloaded_restaurant", "grab_food")
graph.add_edge("manage_overloaded_restaurant", END)
graph.add_edge("damaged_packaging_dispute", "grab_food")
graph.add_edge("damaged_packaging_dispute", END)
graph.add_edge("communicate", "grab_food")
graph.add_edge("communicate", END)

# Set the entry point
graph.set_entry_point("grab_food")

# Compile the graph
app = graph.compile()

ValueError: Found edge starting at unknown node 'communicate'